In [1]:
from google.colab import drive

# Monta o Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install hiddenlayer

In [3]:
!git clone https://github.com/MIC-DKFZ/nnUNet.git

Cloning into 'nnUNet'...
remote: Enumerating objects: 14005, done.
remote: Total 14005 (delta 0), reused 0 (delta 0), pack-reused 14005 (from 1)
Receiving objects: 100% (14005/14005), 8.60 MiB | 11.87 MiB/s, done.
Resolving deltas: 100% (10702/10702), done.


In [4]:
%cd /content

/content


In [5]:
!dir

drive  nnUNet  sample_data


In [6]:
%cd nnUNet

/content/nnUNet


In [7]:
!dir

documentation  LICENSE	nnunetv2  pyproject.toml  readme.md  setup.py


In [8]:
!pip install -e .

Obtaining file:///content/nnUNet
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 121.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 10.0 MB/s eta 0:00:00
  Building editable for nn

In [1]:
import os
os.environ["nnUNet_raw"] = "/content/drive/MyDrive/TCC/1.Data/nnU-net_format/subset_50_PCA/nnUNet_raw"
os.environ["nnUNet_preprocessed"] = "/content/drive/MyDrive/TCC/1.Data/nnU-net_format/subset_50_PCA/nnUNet_preprocessed"
os.environ["nnUNet_results"] = "/content/drive/MyDrive/TCC/1.Data/nnU-net_format/subset_50_PCA/nnUNet_results"

In [2]:
! set nnUNet_raw = /content/drive/MyDrive/TCC/1.Data/nnU-net_format/subset_50/nnUNet_raw
! set nnUNet_preprocessed = /content/drive/MyDrive/TCC/1.Data/nnU-net_format/subset_50/nnUNet_preprocessed
! set nnUNet_results = /content/drive/MyDrive/TCC/1.Data/nnU-net_format/subset_50/nnUNet_results

In [ ]:
import json
from pathlib import Path
import nibabel as nib
import numpy as np

# ------------------- CONFIGURAÇÃO -------------------
base_dir = Path('/content/drive/MyDrive/TCC/1.Data/nnU-net_format/subset_50/nnUNet_raw/Dataset001_MyBraTS')
imagesTr = base_dir / 'imagesTr'
labelsTr = base_dir / 'labelsTr'
imagesTs = base_dir / 'imagesTs'
output_json = base_dir / 'dataset.json'

# ------------------- FUNÇÃO PARA COLETAR LABELS EXISTENTES -------------------
def collect_unique_labels(labels_folder):
    all_labels = set()
    for label_file in labels_folder.glob('*-seg.nii.gz'):
        data = nib.load(label_file).get_fdata()
        unique = np.unique(data)
        all_labels.update(unique.tolist())
    return sorted([int(x) for x in all_labels])

# ------------------- COLETAR TREINO -------------------
train_cases = sorted([f.name.replace('_0000.nii.gz', '') for f in imagesTr.glob('*_0000.nii.gz')])

training = []
for case in train_cases:
    # Procura máscara ignorando capitalização
    matching_label = [f for f in labelsTr.glob('*-seg.nii.gz') if f.stem.lower().startswith(case.lower())]
    if not matching_label:
        print(f"⚠️ Label não encontrada para {case}")
        continue
    label_path = matching_label[0]

    entry = {
        "image": [
            f"./imagesTr/{case}_0000.nii.gz",
            f"./imagesTr/{case}_0001.nii.gz",
            f"./imagesTr/{case}_0002.nii.gz",
            f"./imagesTr/{case}_0003.nii.gz"
        ],
        "label": f"./labelsTr/{label_path.name}"
    }
    training.append(entry)

# ------------------- COLETAR TESTE -------------------
test_cases = sorted([f.name.replace('_0000.nii.gz', '') for f in imagesTs.glob('*_0000.nii.gz')])
test = []
for case in test_cases:
    test.extend([
        f"./imagesTs/{case}_0000.nii.gz",
        f"./imagesTs/{case}_0001.nii.gz",
        f"./imagesTs/{case}_0002.nii.gz",
        f"./imagesTs/{case}_0003.nii.gz"
    ])

# ------------------- COLETAR LABELS EXISTENTES -------------------
unique_labels = collect_unique_labels(labelsTr)
labels_dict = {
    'background': 0,
    'NETC': 1,
    'SNFH': 2,
    'ET': 3,
    'RC': 4,
}

# ------------------- MONTAR JSON -------------------
dataset_json = {
    "name": "Dataset001_MyBraTS",
    "description": "Dataset MyBraTS subset 30 para nnU-Net",
    "reference": "",
    "licence": "",
    "release": "1.0",
    "channel_names": {
        "0": "T1",
        "1": "T1ce",
        "2": "T2",
        "3": "FLAIR"
    },
    "labels": labels_dict,
    "numTraining": len(training),
    "file_ending": ".nii.gz",
    "training": training,
    "test": test
}

# ------------------- SALVAR -------------------
with open(output_json, 'w') as f:
    json.dump(dataset_json, f, indent=4)

print(f"✅ dataset.json criado em: {output_json}")
print(f"Treino: {len(training)} casos | Teste: {len(test_cases)} casos")
print(f"Labels detectadas: {unique_labels}")

✅ dataset.json criado em: /content/drive/MyDrive/TCC/1.Data/nnU-net_format/subset_50/nnUNet_raw/Dataset001_MyBraTS/dataset.json
Treino: 40 casos | Teste: 10 casos
Labels detectadas: [0, 1, 2, 3, 4]


In [ ]:
from nnunetv2.dataset_conversion.generate_dataset_json import generate_dataset_json
from batchgenerators.utilities.file_and_folder_operations import join, maybe_mkdir_p, subdirs
from nnunetv2.paths import nnUNet_raw
from pathlib import Path

# ------------------- CONFIGURAÇÃO -------------------
extracted_dir = Path("/content/drive/MyDrive/TCC/1.Data/nnU-net_format/subset_50_PCA/nnUNet_raw/Dataset001_MyBraTS")
imagesTr = extracted_dir / "imagesTr"
labelsTr = extracted_dir / "labelsTr"
imagesTs = extracted_dir / "imagesTs"

nnunet_dataset_name = "MyBraTS"
nnunet_dataset_id = 1
dataset_name = f"Dataset{nnunet_dataset_id:03d}_{nnunet_dataset_name}"
dataset_dir = join(nnUNet_raw, dataset_name)
maybe_mkdir_p(dataset_dir)

# ------------------- MONTAR DATASET DE TREINO -------------------
train_cases = sorted([f.name.replace(".nii.gz", "").replace("_0000", "") for f in imagesTr.glob("*_0000.nii.gz")])
dataset = {}
for case in train_cases:
    dataset[case] = {
        "label": str(labelsTr / f"{case}-seg.nii.gz"),
        "images": [
            str(imagesTr / f"{case}_0000.nii.gz"),
            str(imagesTr / f"{case}_0001.nii.gz"),
            str(imagesTr / f"{case}_0002.nii.gz"),
        ]
    }

# ------------------- MONTAR DATASET DE TESTE -------------------
test_cases = sorted([f.name.replace(".nii.gz", "").replace("_0000", "") for f in imagesTs.glob("*_0000.nii.gz")])
test_list = []
for case in test_cases:
    test_list.append([
        str(imagesTs / f"{case}_0000.nii.gz"),
        str(imagesTs / f"{case}_0001.nii.gz"),
        str(imagesTs / f"{case}_0002.nii.gz"),
    ])

# ------------------- LABELS E MODALIDADES -------------------
labels_dict = {
    "background": 0,
    "NETC": 1,
    "SNFH": 2,
    "ET": 3,
    "RC": 4,
}

modalities = {
    0: "PCA1",
    1: "PCA2",
    2: "PCA3"
}

# ------------------- GERAR JSON -------------------
generate_dataset_json(
    dataset_dir,
    modalities,
    labels_dict,
    num_training_cases=len(dataset),
    file_ending=".nii.gz",
    regions_class_order=None,
    dataset_name=dataset_name,
    reference="https://www.synapse.org/Synapse:syn53708249/wiki/627500",
    license="see https://www.synapse.org/Synapse:syn53708249/wiki/627508",
    dataset=dataset,
    test=test_list,
    description="Dataset MyBraTS subset 50 para nnU-Net (relative paths, treino + teste separado)."
)

print(f"✅ JSON criado em: {dataset_dir}")
print(f"Treino: {len(train_cases)} casos | Teste: {len(test_cases)} casos")

✅ JSON criado em: /content/drive/MyDrive/TCC/1.Data/nnU-net_format/subset_50_PCA/nnUNet_raw/Dataset001_MyBraTS
Treino: 40 casos | Teste: 10 casos


In [ ]:
!nnUNetv2_plan_and_preprocess -d 1 --verify_dataset_integrity

Fingerprint extraction...
Dataset001_MyBraTS
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer

####################
verify_dataset_integrity Done. 
If you didn't see any error messages then your dataset is most likely OK!
####################

Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer
100% 40/40 [01:02<00:00,  1.56s/it]
Experiment planning...

############################
INFO: You are using the old nnU-Net default planner. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Dropping 3d_lowres config because the image size difference to 3d_fullres is too small. 3d_fullres: [182. 218. 182.], 3d_lowres: [182, 218, 182]
2D U-Net configuration:
{'data_identifier': 'nnUNetPlans_2d', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 75, 'patch_size': (np.int64

In [ ]:
!nnUNetv2_train Dataset001_MyBraTS 3d_fullres 2 -tr nnUNetTrainer_50epochs --npz


############################
INFO: You are using the old nnU-Net default plans. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2025-10-21 18:00:41.352978: Using torch.compile...
2025-10-21 18:00:47.553235: do_dummy_2d_data_aug: False
2025-10-21 18:00:47.590504: Using splits from existing split file: /content/drive/MyDrive/TCC/1.Data/nnU-net_format/subset_50_PCA/nnUNet_preprocessed/Dataset001_MyBraTS/splits_final.json
20